# 언어 모델 미세 조정하기

<!--- @wandbcode{dlai_05} -->

언어 모델을 미세 조정하여 wandb와 통합된 HuggingFace 트레이너를 사용하여 캐릭터 배경 스토리를 생성하는 방법을 살펴 보겠습니다. 여기서는 리소스 제약으로 인해 작은 언어 모델(`TinyStories-33M`)을 사용하지만 여기서 배운 내용은 대규모 모델에도 적용할 수 있을 것입니다!

In [1]:
# !pip install datasets
# !pip install transformers
!pip install accelerate -U
# !pip install transformers[torch]

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import transformers
transformers.set_seed(42)

import wandb

In [3]:
wandb.login(anonymous="allow")

wandb: Currently logged in as: kimwooglae. Use `wandb login --relogin` to force relogin


True

In [4]:
model_checkpoint = "roneneldan/TinyStories-33M"

### 데이터 준비

먼저 허깅페이스에서 던전 앤 드래곤 캐릭터의 약력이 포함된 데이터 세트를 로드하겠습니다.

> You can expect to get some warning here, this is ok

In [5]:
ds = load_dataset('MohamedRashad/characters_backstories')

In [6]:
# Let's take a look at one example
ds["train"][400]

{'text': 'Generate Backstory based on following information\nCharacter Name: Dewin \nCharacter Race: Halfling\nCharacter Class: Sorcerer bard\n\nOutput:\n',
 'target': 'Dewin thought he was a wizard, but it turned out it was the draconic blood in his veins that brought him eldritch power.  Music classes in wizarding college taught him yet another use for his power, and when he was expelled he took up adventuring'}

In [7]:
# As this dataset has no validation split, we will create one
ds = ds["train"].train_test_split(test_size=0.2, seed=42)

In [8]:
# We'll create a tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

# We'll need padding to have same length sequences in a batch
tokenizer.pad_token = tokenizer.eos_token

# Define a tokenization function that first concatenates text and target
def tokenize_function(example):
    merged = example["text"] + " " + example["target"]
    batch = tokenizer(merged, padding='max_length', truncation=True, max_length=128)
    batch["labels"] = batch["input_ids"].copy()
    return batch

# Apply it on our dataset, and remove the text columns
tokenized_datasets = ds.map(tokenize_function, remove_columns=["text", "target"])

In [9]:
# Let's check out one prepared example
print(tokenizer.decode(tokenized_datasets["train"][900]['input_ids']))

Generate Backstory based on following information
Character Name: Mr. Gale
Character Race: Half-orc
Character Class: Cleric

Output:
 Growing up the only half-orc in a small rural town was rough. His mother didn't survive childbirth and so was raised in a church in a high mountain pass, his attention was always drawn by airships passing through, and dreams of an escape. Leaving to strike out on his own as early as he could he made a living for most of his life as an airship sailor, and occasionally a pirate. A single storm visits him throughout his life, marking every major


### 학습

HF Transformers와 wandb 통합을 사용하여 데이터 세트에서 사전 학습된 언어 모델을 미세 조정해 보겠습니다.

In [10]:
# We will train a causal (autoregressive) language model from a pretrained checkpoint
model = AutoModelForCausalLM.from_pretrained(model_checkpoint);

In [11]:
# Start a new wandb run
run = wandb.init(project='dlai_lm_tuning', job_type="training", anonymous="allow")

In [12]:
# Define training arguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-characters-backstories",
    report_to="wandb", # we need one line to track experiments in wandb
    num_train_epochs=1,
    logging_steps=1,
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    no_cuda=True, # force cpu use, will be renamed `use_cpu`
)

In [13]:
# We'll use HF Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [14]:
# Let's train!
trainer.train()

/Users/wlkim/anaconda3/envs/deeplearning-ai/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.942100,3.345123


TrainOutput(global_step=233, training_loss=3.7518335007802612, metrics={'train_runtime': 136.5179, 'train_samples_per_second': 13.603, 'train_steps_per_second': 1.707, 'total_flos': 40423258718208.0, 'train_loss': 3.7518335007802612, 'epoch': 1.0})

In [15]:
transformers.logging.set_verbosity_error() # suppress tokenizer warnings

prefix = "Generate Backstory based on following information Character Name: "

prompts = [
    "Frogger Character Race: Aarakocra Character Class: Ranger Output: ",
    "Smarty Character Race: Aasimar Character Class: Cleric Output: ",
    "Volcano Character Race: Android Character Class: Paladin Output: ",
]

table = wandb.Table(columns=["prompt", "generation"])

for prompt in prompts:
    input_ids = tokenizer.encode(prefix + prompt, return_tensors="pt")
    output = model.generate(input_ids, do_sample=True, max_new_tokens=50, top_p=0.9)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    table.add_data(prefix + prompt, output_text)
    
wandb.log({'tiny_generations': table})

**Note**: LLM이 항상 동일한 결과를 생성하는 것은 아닙니다. 생성된 캐릭터와 배경 스토리는 동영상과 다를 수 있습니다.

In [16]:
wandb.finish()

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▃▂▂▃▄▁▂▁▂▂▂▂▃▃▂▂▂▁▂▂▂▂▃▂▂▂▁▂▂▃▁▁▁▁▂▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
